[Reference](https://bobrupakroy.medium.com/thymeboost-7248297d3165)

In [1]:
pip install ThymeBoost

  Created wheel for ThymeBoost: filename=ThymeBoost-0.1.7-py3-none-any.whl size=46133 sha256=9fcaf68e8c4451122f459269791124b1cec49d760605d4e8851af618153f18cf
  Stored in directory: /root/.cache/pip/wheels/5d/67/89/34b1b84994544149cfb32ab29735b7064f0a933d0f6a9e23a6
Successfully built ThymeBoost


In [2]:
import numpy as np
import pandas as pd 
import searborn as sns
sns.set_style("darkgrid")
from matplotlib import pyplot as plt
from ThymeBoost import ThymeBoost as tb
#load the dataset
df = pd.read_csv('AirPassengers.csv')
df.index = pd.to_datetime(df['Month'])
y = df['Passengers']
#we will split the dataset into train and test
test_len = int(len(y) * 0.3)
al_train, al_test = y.iloc[:-test_len], y.iloc[-test_len:]

In [3]:
boosted_model = tb.ThymeBoost(verbose=0)
output = boosted_model.autofit(al_train,seasonal_period=12)
predicted_output = boosted_model.predict(output, len(al_test)) #al_test* is the length of the future dataset. 
#Here we need to mention the same as test set                            #so that we can compare the predicted and the actual data
#evaluation metrics
tb_mae = np.mean(np.abs(al_test - predicted_output['predictions']))
tb_rmse = (np.mean((al_test - predicted_output['predictions'])**2))**.5
tb_mape = np.sum(np.abs(predicted_output['predictions'] - al_test)) / (np.sum((np.abs(al_test))))
#to extract the components
predicted_output.predicted_seasonality
predicted_output.predicted_trend
#Plot the components
boosted_model.plot_components(output, predicted_output) 

In [4]:
#to extract the components
predicted_output.predicted_seasonality
predicted_output.predicted_trend

In [5]:
#detect outliers
boosted_model = tb.ThymeBoost()
output = boosted_model.detect_outliers(al_train,
                                       trend_estimator='linear',
                                       seasonal_estimator='fourier',
                                       seasonal_period=25,
                                       global_cost='maicc',
                                       fit_type='global')
boosted_model.plot_results(output)
boosted_model.plot_components(output)

In [6]:
#Auto Arima model
import pmdarima as pm

arima = pm.auto_arima(al_train,
                      seasonal=True,
                      m=12,
                      trace=True,
                      error_action='warn',
                      n_fits=50)
pmd_predictions = arima.predict(n_periods=len(al_test))
arima_mae = np.mean(np.abs(al_test - pmd_predictions))
arima_rmse = (np.mean((al_test - pmd_predictions)**2))**.5
arima_mape = np.sum(np.abs(pmd_predictions - al_test)) / (np.sum((np.abs(al_test))))
#Fbprophet
from fbprophet import Prophet
prophet_train_df = al_train.reset_index()
prophet_train_df.columns = ['ds', 'y']
prophet = Prophet(seasonality_mode='multiplicative')
prophet.fit(prophet_train_df)
future_df = prophet.make_future_dataframe(periods=len(al_test), freq='M')
prophet_forecast = prophet.predict(future_df)
prophet_predictions = prophet_forecast['yhat'].iloc[-len(al_test):]
prophet_mae = np.mean(np.abs(al_test - prophet_predictions.values))
prophet_rmse = (np.mean((al_test - prophet_predictions.values)**2))**.5
prophet_mape = np.sum(np.abs(prophet_predictions.values - al_test)) / (np.sum((np.abs(al_test))))

In [7]:
# Creating a dictionary
d = {'mae': [tb_mae,arima_mae,prophet_mae],
     'rmse': [tb_rmse,arima_rmse,prophet_rmse], 
     'mape': [tb_mape,arima_mape,prophet_mape]} 
performance = pd.DataFrame(d,index=["ThymeBoost","Arima","Fbprophet"])